In [1]:
import os
import s3fs
import pandas as pd
import numpy as np

# --- Настройки ---
BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data"
S3_BUCKET = "nyc-tlc-stats"
AWS_REGION = "us-east-1"
LOCAL_DIR = "../nyc_tlc_data"
# CAB_TYPES = ["yellow", "green"]
# MONTHS = ["2025-04", "2025-05", "2025-06", "2025-07", "2025-08", "2025-09"]
CAB_TYPES = ["green"]
MONTHS = ["2025-04"]

os.makedirs(LOCAL_DIR, exist_ok=True)
fs = s3fs.S3FileSystem()


# --- Функции ---
def download_and_upload(cab, month):
    file_name = f"{cab}_tripdata_{month}.parquet"
    local_path = os.path.join(LOCAL_DIR, file_name)
    s3_path = f"s3://{S3_BUCKET}/{file_name}"

    # # Проверяем локально
    # if not os.path.exists(local_path):
    #     url = f"{BASE_URL}/{file_name}"
    #     print(f"Downloading {url} → {local_path}...")
    #     os.system(f"curl -fSL {url} -o {local_path}")
    # else:
    #     print(f"File exists locally: {local_path}")

    # # Проверяем в S3
    # if not fs.exists(s3_path):
    #     print(f"Uploading {local_path} → {s3_path}...")
    #     os.system(f"aws s3 cp {local_path} {s3_path} --region {AWS_REGION}")
    # else:
    #     print(f"File already exists in S3: {s3_path}")

    return local_path


# --- Загрузка и подготовка всех файлов ---
all_dfs = []
for cab in CAB_TYPES:
    for month in MONTHS:
        local_file = download_and_upload(cab, month)
        df_month = pd.read_parquet(local_file)
        df_month["cab_type"] = cab
        all_dfs.append(df_month)

# --- Объединение, очистка и фильтрация ---
df = pd.concat(all_dfs, ignore_index=True)
df.info()
df.head()

# # Преобразование дат
# df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], errors="coerce")
# df["tpep_dropoff_datetime"] = pd.to_datetime(
#     df["tpep_dropoff_datetime"], errors="coerce"
# )
# df = df.dropna(subset=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# # Числовые колонки
# numeric_cols = [
#     "passenger_count",
#     "trip_distance",
#     "fare_amount",
#     "total_amount",
#     "PULocationID",
#     "DOLocationID",
# ]
# df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")
# df = df.dropna(subset=numeric_cols)
# df["passenger_count"] = df["passenger_count"].astype(int)
# df["PULocationID"] = df["PULocationID"].astype(int)
# df["DOLocationID"] = df["DOLocationID"].astype(int)


# # Фильтрация выбросов
# df = df.query(
#     "1 <= passenger_count <= 9 & 0 < trip_distance < 100 & 0 < fare_amount < 1000 & 0 < total_amount < 1000"
# )

# # Добавляем driver_id и trip_time
# np.random.seed(42)
# df["driver_id"] = np.random.randint(1, 101, size=len(df))
# df["trip_time_min"] = (
#     df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
# ).dt.total_seconds() / 60

# # Итоговый DataFrame для ClickHouse
# df_clickhouse_clean = df[
#     [
#         "cab_type",
#         "tpep_pickup_datetime",
#         "tpep_dropoff_datetime",
#         "driver_id",
#         "passenger_count",
#         "trip_distance",
#         "PULocationID",
#         "DOLocationID",
#         "fare_amount",
#         "total_amount",
#         "trip_time_min",
#     ]
# ]

# print(df_clickhouse_clean.info())
# print(df_clickhouse_clean.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52132 entries, 0 to 52131
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               52132 non-null  int32         
 1   lpep_pickup_datetime   52132 non-null  datetime64[us]
 2   lpep_dropoff_datetime  52132 non-null  datetime64[us]
 3   store_and_fwd_flag     48944 non-null  object        
 4   RatecodeID             48944 non-null  float64       
 5   PULocationID           52132 non-null  int32         
 6   DOLocationID           52132 non-null  int32         
 7   passenger_count        48944 non-null  float64       
 8   trip_distance          52132 non-null  float64       
 9   fare_amount            52132 non-null  float64       
 10  extra                  52132 non-null  float64       
 11  mta_tax                52132 non-null  float64       
 12  tip_amount             52132 non-null  float64       
 13  t

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,cab_type
0,2,2025-04-01 00:36:05,2025-04-01 00:42:00,N,1.0,75,41,1.0,1.37,8.6,...,1.00,0.0,NaN,1.0,12.10,1.0,1.0,0.00,0.0,green
1,2,2025-04-01 00:10:24,2025-04-01 00:30:29,N,1.0,74,143,1.0,3.99,21.9,...,5.43,0.0,NaN,1.0,32.58,1.0,1.0,2.75,0.0,green
2,2,2025-04-01 00:13:12,2025-04-01 00:26:10,N,1.0,260,129,1.0,2.37,14.9,...,0.00,0.0,NaN,1.0,17.40,2.0,1.0,0.00,0.0,green
3,2,2025-04-01 00:15:37,2025-04-01 00:38:07,N,1.0,130,7,1.0,10.40,42.2,...,11.18,0.0,NaN,1.0,55.88,1.0,1.0,0.00,0.0,green
4,2,2025-04-01 00:14:27,2025-04-01 00:24:08,N,1.0,244,166,2.0,2.98,14.9,...,0.00,0.0,NaN,1.0,17.40,2.0,1.0,0.00,0.0,green
